Connect ADLS Gen2 for Email Automation project

In [0]:
storage_account_name = "rupamemailautomation "
client_id = dbutils.secrets.get(scope = "databricks-scope", key = "databricks-app-client-id")
tenant_id = dbutils.secrets.get(scope = "databricks-scope", key = "databricks-app-tenant-id")
client_secret = dbutils.secrets.get(scope = "databricks-scope", key = "databricks-app-client-secret")

In [0]:
configs = {
    "fs.azure.account.auth.type": "OAuth",
    "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
    "fs.azure.account.oauth2.client.id": f"{client_id}",
    "fs.azure.account.oauth2.client.secret": f"{client_secret}",
    "fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{tenant_id}/oauth2/token",
    "fs.azure.createRemoteFileSystemDuringInitialization": "true",
    "fs.azure.permissions.blob.directory.rupamemailautomation.users.rwx": "true"
}

In [0]:
def mount_adls(container_name, storage_account_name):
    dbutils.fs.mount(
        source = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/",
        mount_point = f"/mnt/{storage_account_name}/{container_name}",
        extra_configs = configs
    )

In [0]:
def call_mount_adls_if_already_not_mounted(container_name, storage_account_name):
    mounts = dbutils.fs.mounts()
    storage_account_name_arr = []
    container_name_arr = []

    for urls in mounts:
        # print(urls.mountPoint[:4])
        if urls.mountPoint[:4] == "/mnt":
            storage_account_name_arr.append(urls.mountPoint.split("/")[2])
            # print(urls.mountPoint.split("/")[2])
            container_name_arr.append(urls.mountPoint.split("/")[3])
            # print(urls.mountPoint.split("/")[3])

    # print(storage_account_name_arr)
    # print(container_name_arr)
    if container_name not in container_name_arr or storage_account_name not in storage_account_name_arr:
        mount_adls(container_name, storage_account_name)


In [0]:
call_mount_adls_if_already_not_mounted("managersdata", "rupamemailautomation")

call_mount_adls_if_already_not_mounted("employeesdata", "rupamemailautomation")

In [0]:
dbutils.fs.ls("/mnt/rupamemailautomation")

In [0]:
# dbutils.fs.ls("/mnt/rupamemailautomation/managersdata")

In [0]:
# df = spark.read \
#   .option("inferSchema" , True,) \
#   .option("header", True) \
#   .csv("dbfs:/mnt/rupamemailautomation/managersdata/manager.two_03.04.23.csv")

# display(df)

In [0]:
# dbutils.fs.unmount("/mnt/rupamemailautomation/managersdata")

# dbutils.fs.unmount("/mnt/rupamemailautomation/employeesdata")

In [0]:
dbutils.notebook.exit("success")